In [1]:
from recordclass import dataobject, enable_gc, clsconfig
import matplotlib.pyplot as plt
import numpy as np
import time
import gc

In [2]:
class Point(dataobject):
    x:int
    y:int
    __options__ = {'argsonly':True}

@enable_gc
class Point_GC(dataobject):
    x:int
    y:int
    __options__ = {'argsonly':True}
        
class Point_Slots:
    __slots__ = 'x','y','c','d','e','f'
    x:int
    y:int
        
    def __init__(self, x, y):
        self.x = x; self.y = y

In [3]:
def test(N):
#     ll = N*[None]
    for i in range(N):
        c = Point(1,2)
#         ll[i] = c
        
def test_gc(N):
#     ll = N*[None]
    for i in range(N):
        c = Point_GC(1,2)
#         ll[i] = c
        
def test_slots(N):
#     ll = N*[None]
    for i in range(N):
        c = Point_Slots(1,2)
#         ll[i] = c

In [ ]:
gc.collect()
m = 22
Ns = [int(2**x) for x in range(1,m+1)]
Ts = np.zeros(m)
Ts_gc = np.zeros(m)
Ts_slots = np.zeros(m)
for i in range(m):
    N = Ns[i]

    gc.collect()
    gc.collect()

    t0 = time.time()
    test(N)
    Ts[i] = time.time()-t0
    
    gc.collect()
    gc.collect()
    
    t1 = time.time()
    test_gc(N)
    Ts_gc[i] = time.time()-t1

#     gc.collect()
#     gc.collect()

#     t1 = time.time()
#     test_slots(N)
#     Ts_slots[i] = time.time()-t1
    

In [ ]:
plt.figure(figsize=(8, 6))
# plt.plot(Ns, Ts, marker='o', label='nogc')
plt.plot(Ns, Ts_gc/Ts, marker='o', label='gc')
# plt.loglog(Ns, Ts_slots, marker='s', label='slots')
plt.legend()
plt.xlabel("Number of instances")
plt.ylabel('Time')
plt.grid(1)
plt.show()

In [ ]:
import pickle

In [ ]:
p1 = Point(1,2)
p2 = Point_Slots(1, 2)

In [ ]:
b1 = pickle.dumps(p1)
b2 = pickle.dumps(p2)

In [ ]:
print(len(b1), len(b2))

In [ ]:
print(b1)
print(b2)

In [ ]:
print(pickle.loads(b1))

In [ ]:
%timeit pickle.loads(pickle.dumps([Point(1,2) for i in range(1000)]))

In [ ]:
%timeit pickle.loads(pickle.dumps([Point_Slots(1,2) for i in range(1000)]))